<h1 style="text-align:center"> 资本市场图谱-答案 </h1>
<h3 style="text-align:center">  ——武汉大学金融科技研究中心 </h3>
<h3 style="text-align:center">  ——武汉大学《金融科技》课程作业 </h3>

知识图谱是一种大规模语义网络，以实体或者概念作为节点，通过语义关系相连接，通过发掘实体之间的关联，将结构化、半结构化、非结构化的数据进行整合。它可以帮助理解数据、解释数据、推理数据，从而发掘深层关系实现智慧搜索和智能交互。     
通过构建资本市场知识图谱可以进行可视化展示，将庞大复杂的数据通过图谱的方式进行展现，可以帮助我们更好的理解和应用数据；可以进行关联分析，寻找并构建某一实体的关联图谱；还可以实现对市场数据及基本面的分析，帮助我们进行更全面、更即时的投研。    
本实验我们将基于一些股票数据、上市公司和管理层数据和公募基金数据，搭建一个资本市场知识图谱，并实现简单的信息可视化查询、关联分析和推荐等功能。

## 1 数据介绍

本实验提供以下 6 类数据，这些数据互有关联，互相依赖形成最终的金融知识图谱。
- 公司列表：主要包含了全A股上市公司的证券代码、证券简称、公司全称、行业名称等相关信息。
- 公司明细：主要包含了上市公司地理位置等具体的信息。
- 公司管理者：主要包含了上市公司各董监会人员的姓名、性别、学历等相关信息。
- 基金基本信息，基金公司信息：主要包含了基金代码、基金公司名称、管理人、托管人等相关信息。
- 基金持仓：主要包含了2019年以来公告的所有公募基金季度更新的持仓数据。

In [1]:
import pandas as pd

In [2]:
data_path = './data/'# 将本实验提供的数据放置在工作区下的该文件夹中

### 1.1 公司列表
主要包含了全A股上市公司的证券代码、证券简称、公司全称、行业名称等相关信息。

|名称|类型|描述|
|--|--|--|
|ts_code|str|TS代码|
|symbol|str|<font  color=#009688  >**股票代码**</font>|
|name|str|<font  color=#009688  >**股票名称**</font>|
|industry|str|<font  color=#009688  >**所属行业（以2012版证监会行业分类为准）**</font>|
|fullname|str|<font  color=#009688  >**股票全称**</font>|
|market|str|市场类型：上海A/上海B/深圳A/深圳B/创业板/科创板|
|exchange|str|交易所代码：SSE上交所 SZSE深交所|
|curr_type|str|交易货币|
|list_status|str|上市状态： L上市 D退市 P暂停上市|
|list_date|str|上市日期|
|delist_date|str|退市日期|

In [3]:
usecols = ['ts_code','symbol','name','fullname','market','exchange','list_status',
           'list_date','delist_date']

df_company_lsit_all =  pd.read_csv(data_path + '公司列表.csv',dtype={"symbol":str},usecols=usecols, encoding = 'ansi')

**<font color = red>问题一：“公司列表”数据缺乏上市公司所在的行业信息。请你以2012年版证监会行业分类为准，从CSMAR数据库中下载数据补充该部分信息作为industry列。</font>**

In [4]:
hy = pd.read_csv(data_path +'TRD_Co.csv')
df_company_lsit_all = pd.merge(df_company_lsit_all, hy[['Stkcd', 'Nnindnme']], left_on = 'symbol', right_on = 'Stkcd')
df_company_lsit_all = df_company_lsit_all.rename(columns = {'Nnindnme': 'industry'})
df_company_lsit_all.head()

,ts_code,symbol,name,industry,fullname,market,exchange,list_status,list_date,delist_date
0,000001.SZ,000001,平安银行,货币金融服务,平安银行股份有限公司,深圳A,SZSE,L,19910403,NaN
1,000002.SZ,000002,万科A,房地产业,万科企业股份有限公司,深圳A,SZSE,L,19910129,NaN
2,000004.SZ,000004,国农科技,软件和信息技术服务业,深圳中国农大科技股份有限公司,深圳A,SZSE,L,19910114,NaN
3,000005.SZ,000005,世纪星源,生态保护和环境治理业,深圳世纪星源股份有限公司,深圳A,SZSE,L,19901210,NaN
4,000006.SZ,000006,深振业A,房地产业,深圳市振业(集团)股份有限公司,深圳A,SZSE,L,19920427,NaN


**<font color = red>问题二：基于“公司列表”数据，统计截止2019年底为止，A股市场总共有多少家上市公司，其中在上交所上市的有多少家，请你补充所需要的代码到下面一个代码块，并打印输出结果：</font>**

In [5]:
df_company_lsit_all0 = df_company_lsit_all[(df_company_lsit_all['list_date'] < 20200000) & (df_company_lsit_all['list_status'] == 'L')]
df_company_lsit_all1 = df_company_lsit_all0[(df_company_lsit_all0['market']=='上海A') | (df_company_lsit_all0['market']=='深圳A' )|( df_company_lsit_all0['market']=='创业板' )|( df_company_lsit_all0['market']=='科创板')]
print('截止2019年底为止，A股市场上市公司数目:', len(df_company_lsit_all1))
df_company_lsit_all2 = df_company_lsit_all1[df_company_lsit_all1['exchange']=='SSE']
len(df_company_lsit_all2)
print('其中在上交所上市的公司数目:', len(df_company_lsit_all1))

截止2019年底为止，A股市场上市公司数目: 3730
其中在上交所上市的公司数目: 3730


### 1.2 公司明细
这部分数据主要包含了上市公司地理位置等具体的信息，将用于提取上市公司所在省份和城市。

|名称|类型|描述|
|---|---|---|
|ts_code|str| <font  color=#009688  >**股票代码**</font>|
|exchange|str| 交易所代码 ，SSE上交所 SZSE深交所|
|chairman|str| 法人代表|
|manager |str| 总经理|
|secretary| str| 董秘|
|reg_capital |float|  注册资本|
|setup_date|str| 注册日期|
|province|str| <font  color=#009688  >**所在省份**</font>|
|city|str|  <font  color=#009688  >**所在城市**</font>|
|introduction|str| 公司介绍|
|website |str| 公司主页|
|email| str| 电子邮件|
|office|str| 办公室|
|employees| int| 员工人数|
|main_business| str| 主要业务及产品|
|business_scope|str| 经营范围|

In [123]:
df_company_detail = pd.read_csv(data_path + '公司明细.csv', encoding = 'ansi')
df_company_detail.head()

,ts_code,exchange,chairman,manager,secretary,reg_capital,setup_date,province,city,website,email,employees
0,300304.SZ,SZSE,付红玲,蔡承儒,郑渲薇,86603.6018,20070427,江苏,徐州市,www.yunyi-china.com,dsh@yunyi-china.com,948.0
1,002635.SZ,SZSE,吕莉,吕莉,马玉燕,63420.2080,19991216,江苏,苏州市,www.anjiesz.com,zhengquan@anjiesz.com,6391.0
2,002836.SZ,SZSE,肖海兰,肖海兰,夏明珠,16000.0000,20060322,广东,潮州市,www.newglp.com,gdxhz@newglp.com;xiamz@newglp.com;zhendai@newg...,689.0
3,002707.SZ,SZSE,冯滨,冯滨,郭镭,90994.5822,19920811,北京,北京市,www.uzai.com,stock@utourworld.com,5468.0
4,002629.SZ,SZSE,温志平,陈曦,王晶,41194.8000,20060927,浙江,温州市,www.chinarenzhi.com,ofc_board@renzhi.cn,137.0


### 1.3 公司管理者
这部分数据主要包含了上市公司各董监会人员的姓名、性别、学历等相关信息，将用于提取上市公司各董监会人员的姓名。

|名称|类型|描述|
|----|----|--|
|ts_code| str| <font  color=#009688  >**TS股票代码**</font>|
|ann_date|str| 公告日期|
|name|str| <font  color=#009688  >**姓名**</font>|
|gender|str| 性别|
|lev|str| 岗位类别|
|title| str| 岗位|
|edu|str| 学历|
|national|str| 国籍|
|birthday|str| 出生年月|
|begin_date|str| 上任日期|
|end_date|str| 离任日期|

In [124]:
df_stk_managers = pd.read_csv(data_path + '公司管理者.csv', encoding = 'ansi')

**<font color = red>问题三：基于“公司列表”数据和“公司明细”数据，检阅下万科A(000002.SZ)的信息，提取出该股票的股票全称、市场类型、所在省份、所在城市以及其现任董监会人员的任职情况、姓名、性别、学历等相关信息，请你补充所需要的代码到下面一个代码块，并打印输出结果：</font>**

In [10]:
wanke1 = df_company_lsit_all[df_company_lsit_all['ts_code'] == '000002.SZ'][['symbol','industry','fullname','market']].reset_index()
wanke2 = df_company_detail[df_company_detail['ts_code'] == '000002.SZ'][['province','city']].reset_index()
print(pd.concat([wanke1, wanke2], axis = 1))
wanke3 = df_stk_managers[(df_stk_managers['ts_code'] == '000002.SZ') & (df_stk_managers['end_date'].isna())][['name','gender','lev','title','edu']]
print(wanke3)

   index  symbol industry    fullname market  index province city
0      1  000002     房地产业  万科企业股份有限公司    深圳A   1095       广东  深圳市
       name gender    lev      title edu
262710  王文金      M     高管      财务负责人  硕士
262715  吉江华      M     其他  董事会证券事务代表  硕士
262717  祝九胜      M     高管      首席执行官  博士
262718  祝九胜      M     高管         总裁  博士
262721  王文金      M     高管      执行副总裁  硕士
262722   解冻      M     监事      监事会主席  博士
262723  陈贤军      M  董事会成员         董事  硕士
262724  孙盛典      M  董事会成员         董事  博士
262725   张旭      M  董事会成员         董事  硕士
262726  刘姝威      F  董事会成员       独立董事  硕士
262727  吴嘉宁      M  董事会成员       独立董事  硕士
262728   李强      M  董事会成员       独立董事  硕士
262729   李强      M     其他    投资委员会委员  硕士
262730   李强      M     其他   投资委员会召集人  硕士
262731  吴嘉宁      M     其他    审计委员会委员  硕士
262732  吴嘉宁      M     其他   审计委员会召集人  硕士
262733  陈贤军      M     其他    审计委员会委员  硕士
262734  孙盛典      M     其他    投资委员会委员  博士
262735   张旭      M     其他    投资委员会委员  硕士
262736   康典      M     其他    提名委员会委员  硕士
262737 

### 1.4 基金基本信息
这部分数据主要包含了基金代码、简称、管理人、托管人等相关信息。

|名称|类型|描述|
|--|--|--|
|ts_code|str|<font  color=#009688  >**基金代码**</font>|
|name|str|简称|
|management|str|<font  color=#009688  >**管理人**</font>|
|custodian|str|<font  color=#009688  >**托管人**</font>|
|fund_type|str|投资类型|
|found_date|str|成立日期|
|due_date|str|到期日期|
|list_date|str|上市时间|
|issue_date|str|发行日期|
|delist_date|str|退市日期|
|issue_amount|float|发行份额(亿)|
|m_fee|float|管理费|
|c_fee|float|托管费|
|duration_year|float|存续期|
|p_value|float|面值|
|min_amount|float|起点金额(万元)|
|exp_return|float|预期收益率|
|benchmark|str|业绩比较基准|
|status|str|存续状态D摘牌 I发行 L已上市|
|invest_type|str|投资风格|
|type|str|基金类型|
|market|str|E场内O场外|

In [11]:
df_fund_basic = pd.read_csv(data_path + '基金基本信息.csv', encoding = 'ansi')

In [12]:
df_fund_basic.shape

(11187, 22)

In [13]:
df_fund_basic.head()

,ts_code,name,management,custodian,fund_type,found_date,due_date,list_date,issue_date,delist_date,...,c_fee,duration_year,p_value,min_amount,exp_return,benchmark,status,invest_type,type,market
0,159821.SZ,BOCI创业,中银证券,中国工商银行,股票型,20200929,NaN,20201029.0,20200914.0,NaN,...,0.05,NaN,1.0,0.1,NaN,创业板指数收益率,L,被动指数型,契约型开放式,E
1,159822.SZ,新经济,银华基金,中国工商银行,股票型,20200929,NaN,20201023.0,20200911.0,NaN,...,0.10,NaN,1.0,0.1,NaN,标普中国新经济行业(A股上限)指数收益率,L,被动指数型,契约型开放式,E
2,588080.SH,科创板50ETF,易方达基金,中国工商银行,股票型,20200928,NaN,NaN,20200922.0,NaN,...,0.10,NaN,1.0,0.1,NaN,上证科创板50成份指数收益率,I,被动指数型,契约型开放式,E
3,588090.SH,科创板ETF,华泰柏瑞基金,中国建设银行,股票型,20200928,NaN,NaN,20200922.0,NaN,...,0.10,NaN,1.0,0.1,NaN,上证科创板50成份指数收益率,I,被动指数型,契约型开放式,E
4,511000.SH,长三角地方债ETF,招商基金,浙商银行,债券型,20200928,NaN,20201029.0,20200622.0,NaN,...,0.05,NaN,1.0,0.1,NaN,中债-0-3年长三角地方政府债指数收益率,L,被动指数型,契约型开放式,E


### 1.5 基金公司信息
这部分数据主要包含了基金公司名称、简称、省份、城市等相关信息。

|名称|类型|描述|
|--|--|--|
|name|str|<font  color=#009688  >**基金公司名称**</font>|
|shortname|str|简称|
|province|str|省份|
|city|str|城市|
|address|str|注册地址|
|office|str|办公地址|
|website|str|公司网址|
|setup_date|str|成立日期|
|end_date|str|公司终止日期|
|main_business|str|主要产品及业务|

In [14]:
df_fund_company = pd.read_csv(data_path + '基金公司信息.csv', encoding = 'ansi')

In [15]:
df_fund_company.head().T

,0,1,2,3,4
name,北京广能投资基金管理有限公司,宏源证券股份有限公司,国元证券股份有限公司,广发证券股份有限公司,长江证券股份有限公司
shortname,广能基金,宏源证券,国元证券,广发证券,长江证券
province,北京,新疆,安徽,广东,湖北
city,北京市,乌鲁木齐市,合肥市,广州市,武汉市
address,北京市朝阳区北四环中路27号院5号楼2712-2715A,新疆维吾尔自治区乌鲁木齐市文艺路233号宏源大厦,安徽省合肥市梅山路18号,广东省广州市黄埔区中新广州知识城腾飞一街2号618室,湖北省武汉市江汉区新华路特8号
phone,NaN,86-991-2301870,"86-551-62207323,86-551-62207968","86-20-87555888,86-20-87550565,86-20-87550265","86-27-65799866,86-27-65799856"
office,北京市朝阳区北四环中路27号院5号楼2712-2715A,新疆维吾尔自治区乌鲁木齐市文艺路233号宏源大厦,安徽省合肥市梅山路18号,"广东省广州市天河区天河北路183-187号大都会广场40楼5楼,7楼,8楼,18楼,19楼,...",湖北省武汉市江汉区新华路特8号
website,www.gnfund.cn,www.hysec.com,www.gyzq.com.cn,www.gf.com.cn,www.cjsc.com
chairman,刘锡潜,冯戎,蔡咏,孙树明,尤习贵
manager,杨运成,冯戎,俞仕新,林治海,刘元瑞


### 1.6 基金持仓
这部分数据主要包含了2019年以来公告的所有公募基金季度更新的持仓信息，通过它可以将基金和股票联系起来。

|名称|类型|描述|
|--|--|--|
|ts_code| str| <font  color=#009688  >**基金代码**</font>|
|ann_date|str | 公告日期|
|end_date|str | 截止日期|
|symbol|str|   <font  color=#009688  >**股票代码**</font>|

In [18]:
df_fund_portfolio = pd.read_csv(data_path + '基金持仓.csv', encoding = 'ansi').iloc[:,1:]
df_fund_portfolio.head()

,ts_code,ann_date,end_date,symbol,mkv,amount,stk_mkv_ratio,stk_float_ratio
0,150032.SZ,20160420,20160331,000998.SZ,993524.0,54800.0,10.36,0.01
1,150032.SZ,20160420,20160331,601088.SH,1782808.0,126800.0,18.59,0.00
2,150032.SZ,20160420,20160331,300203.SZ,1119971.4,42747.0,11.68,0.01
3,150032.SZ,20160420,20160331,300012.SZ,307703.1,14590.0,3.21,0.00
4,150032.SZ,20160420,20160331,600900.SH,520672.0,42400.0,5.43,0.00


**<font color = red>问题四：基于“基金持仓”数据，按照持股市值排序降序列出在2010年第二季度公告持有万科A(000002.SZ)的基金，并列出基金代码、基金简称、持有市值、基金管理人和托管人等信息。请你补充所需要的代码到下面一个代码块，并打印输出结果：</font>**

In [19]:
df_fund_portfolio1 = df_fund_portfolio[(df_fund_portfolio['symbol'] == '000002.SZ') & (df_fund_portfolio['end_date'] == 20100630)][['ts_code','mkv']].sort_values(by = 'mkv', ascending = False)
df_fund_basic1 = df_fund_basic[df_fund_basic['ts_code'].isin(list(df_fund_portfolio1['ts_code']))][['ts_code','name', 'management', 'custodian']] 
print(pd.merge(df_fund_portfolio1, df_fund_basic1, on = 'ts_code'))

      ts_code           mkv    name management custodian
0   160505.SZ  3.525594e+08    博时主题       博时基金    中国建设银行
1   160716.SZ  5.899776e+07    嘉实50       嘉实基金    中国工商银行
2   162307.SZ  3.113594e+07   海富100      海富通基金      中国银行
3   519918.OF  2.951847e+07    华夏兴和       华夏基金    中国建设银行
4   160627.OF  1.368416e+07  鹏华策略优选       鹏华基金    中国工商银行
5  1618111.SZ  7.385081e+06   银华300       银华基金    中国建设银行
6   519909.OF  6.780000e+04    华安安顺       华安基金      交通银行


**<font color = red>问题五：请找出江苏省最受基金欢迎的10家上市公司，列出它们所在的城市并统计持有它们的基金的数目。请你补充所需要的代码到下面一个代码块，并打印输出结果：</font>**

In [30]:
df_company_detail1 = df_company_detail[['ts_code', 'province']].rename(columns = {'ts_code': 'symbol'})
df_fund_portfolio1 = pd.merge(df_fund_portfolio, df_company_detail1, on = ['symbol'])
df_fund_portfolio1 = df_fund_portfolio1.drop_duplicates(subset = ['ts_code','symbol'], keep = 'first')
top10 = pd.DataFrame(df_fund_portfolio1['symbol'].value_counts()).rename(columns = {'symbol': 'count'})
top10 = pd.merge(top10, df_company_detail[['ts_code', 'province', 'city']], left_index = True, right_on = 'ts_code')
top10 = top10[top10['province'] == '江苏'].iloc[0:10,:]
top10

,count,ts_code,province,city
901,1216,300842.SZ,江苏,无锡市
3397,1144,600276.SH,江苏,连云港市
3984,963,601688.SH,江苏,南京市
727,958,300782.SZ,江苏,无锡市
4157,841,688377.SH,江苏,南京市
4001,772,603259.SH,江苏,无锡市
2098,702,002304.SZ,江苏,宿迁市
2729,674,601009.SH,江苏,南京市
2700,653,603655.SH,江苏,常州市
2711,652,603283.SH,江苏,苏州市


## 2 构建数据模型 


在资本市场中，根据不同的公司类型，我们可以将公司分为上市公司、基金管理公司以及基金托管公司；根据不同的职能我们可以将公司的管理者类型进行划分；此外资本市场还有各种各样的基金、各种行业类型，现实生活中我们还有省份和城市这样的区位属性。这些都可以作为我们资本市场中的实体进行研究。我们对以上数据进行梳理，基于其内部的关联关系，确定以下节点和关系以构建数据模型。

**实体(节点)：**
1. 公司(上市公司/基金管理公司/基金托管公司)
2. 行业
3. 省份
4. 城市
5. 人（上市公司董监会人员）
6. 基金

**关系：**
1. 根据城市和省份本身的对应关系可以得到：城市 - [IN_PROVINCE] → 省份 
2. 根据各公司的地理信息可以得到：公司(上市公司/基金管理公司/基金托管公司) - [IN_CITY] → 城市
4. 根据上市公司的董监会人员信息可以得到：公司(上市公司) - [HAS_MANEGER] → 人（上市公司管理层）  [6类]
5. 根据上市公司基本信息可以得到：公司(上市公司) - [IN_INDUSTRY] → 行业
6. 根据基金的基本信息可以得到：基金  - [HAS_MANAGEMNET] → 公司(基金管理公司)
6. 根据基金的基本信息可以得到：基金  - [HAS_CUSTODIAN] → 公司(基金托管公司)
7. 根据基金的持仓数据可以得到：基金  - [IN_PORTFOLIO] →  公司(上市公司)

基于以上节点和关系，我们进行数据抽取。

**<font color = red>问题六：请你绘制一张资本市场数据模型简图，标记出以上实体和关系，并且标记出它们是通过什么字段进行联系的。请在下方位置插入图片：</font>**

![title](img/网络图.jpg)

In [31]:
output_path = './data/output/' # 该文件夹将用于放置数据抽取得到的相关节点和关系文件

In [32]:
all_dicts ={}

## 2.1 实体
在资本市场中，根据不同的公司类型，我们可以将公司分为上市公司、基金管理公司以及基金托管公司；根据不同的职能我们可以将公司的管理者类型进行划分；此外资本市场还有各种各样的基金、各种行业类型，现实生活中我们还有省份和城市这样的区位属性。这些都可以作为我们资本市场中的实体进行研究。
### 2.1.1 公司
公司这个实体包含三类数据，分别是上市公司、基金管理公司以及基金托管公司。我们首先需要将这三类公司的相关数据做合并处理，再进行数据抽取。
#### 2.1.1.1 上市公司
* **为上市公司建立标签**

In [33]:
df_company_lsit_all['list_date'] = df_company_lsit_all['list_date'].apply(lambda x: -1 if pd.isnull(x) else int(x) )# 处理日期
df_company_lsit_all['delist_date'] = df_company_lsit_all['delist_date'].apply(lambda x: -1 if pd.isnull(x) else int(x) )# 处理日期

#建立标签
df_company_lsit_all['label_listed_company']='LISTED_COMPANY'

#### 2.1.1.2 基金管理公司
*  **处理并补充基金管理公司信息**  

我们发现fund_company表中的基金管理公司（name）和fund_basic表中的基金管理人（management）有五个银行、一个券商和一个资管名称公司简称并**没有对齐**，这会带来后续的问题。

因此，需要我们手工对齐：

**<font color = red>问题七：现在这7个没有对齐的实体里有两个是空缺的，实体没有对齐可能会引发后续的报错，请你根据报错提示找到剩下的两个实体，并将其对齐，填充到下面的词典中：</font>**

In [34]:
comp_simplify_dict = {
    '中国民生银行':'民生银行',
    '中国光大银行':'光大银行', 
    '中国工商银行':'工商银行', 
    '中国建设银行':'建设银行',
    '中国农业银行':'农业银行',
    '中银国际证券':'中银证券',
    '财通证券资管':'财通资管'
}

此外，fund_basic表里有18个基金管理人（management）没有在fund_company表中出现，需要额外补全其全称。

**<font color = red>问题八：现在fund_company表中的18个缺失的里有6个是空缺的，名称的缺失可能会引发后续的报错，请你根据报错提示找到缺失的其他6个基金管理人，并查询对应的信息，填充到下面的词典中：</font>**

In [35]:
# 基金管理人补齐全称
fund_manage_fullname_dict = {
    '东证资管':'东方证券资产管理有限公司',
    '中泰资管':'中泰证券（上海）资产管理有限公司',
    '华泰证券资管':'华华泰证券资产管理公司',
    '华融基金':'华融基金管理有限公司',
    '博远基金':'博远基金管理有限公司',
    '同泰基金':'同泰基金管理有限公司',
    '平安基金':'平安基金管理有限公司',
    '惠升基金':'惠升基金管理有限公司',
    '朱雀基金':'朱雀基金管理有限公司',
    '淳厚基金':'淳厚基金管理有限公司',
    '睿远基金':'睿远基金管理有限公司',
    '西藏东财基金':'西藏东财基金管理有限公司',
    '山西证券':'山西证券股份有限公司',
    '恒丰银行':'恒丰银行股份有限公司',
    '广发银行':'广发银行股份有限公司',
    '兴证全球基金':'兴证全球基金管理有限公司',
    '达诚基金':'达诚基金管理有限公司',
    '明亚基金':'明亚基金管理有限责任公司'
} 

基于以上信息处理得到新的fund_company表中的基金管理人数据：

In [36]:
usecols = ['name','shortname','province','city','address','office','website',
            'setup_date','end_date','main_business','org_code','credit_code']
df_fund_company = df_fund_company[usecols]

**<font color = red>问题九：基于前面我们找到的基金管理人缺漏词典，我们可以对df_fund_company数据框里面的'name'这一列进行补充，得到完整的基金管理人全称数据，请你补充所需要的代码到下面一个代码块的最后一行：</font>**

In [37]:
df_fund_company['setup_date'] = df_fund_company['setup_date'].apply(lambda x: -1 if pd.isnull(x) else int(x) )
df_fund_company['end_date'] = df_fund_company['end_date'].apply(lambda x: -1 if pd.isnull(x) else int(x) )
df_fund_company = df_fund_company.rename(columns ={'name':'fullname','shortname':'name'}).drop_duplicates()
df_fund_company['name'] = df_fund_company['name'].apply(lambda x : comp_simplify_dict[x] if x in comp_simplify_dict.keys() else x )

基于以上信息处理得到新的fund_basic表中的基金管理公司数据：

In [38]:
df_fund_basic['management'] = df_fund_basic['management'].apply(lambda x : comp_simplify_dict[x] if x in comp_simplify_dict.keys() else x )
df_fund_basic['custodian'] = df_fund_basic['custodian'].apply(lambda x : comp_simplify_dict[x] if x in comp_simplify_dict.keys() else x )

In [39]:
# 抽取df_fund_basic中基金管理人management的实体信息
node_fund_management = df_fund_basic.loc[df_fund_basic.management.notnull(),['management']]
node_fund_management = node_fund_management.rename(columns = {'management':'name'})

* **合并基金管理公司数据，为基金管理公司建立实体标签**

In [40]:
df_fund_company = pd.merge(df_fund_company,node_fund_management,on=['name'],how='outer').drop_duplicates()
df_fund_company['fullname'] = df_fund_company.apply(lambda x:fund_manage_fullname_dict[x['name']] if pd.isnull(x['fullname']) else x['fullname'],axis=1 )
df_fund_company['label_manager'] = 'FUND_MANAGER'

#### 2.1.1.3 基金托管公司  

* **为基金托管公司建立实体标签**

In [41]:
node_fund_custodian = df_fund_basic.loc[df_fund_basic.custodian.notnull(),['custodian']].drop_duplicates()
node_fund_custodian = node_fund_custodian.rename(columns = {'custodian':'name'})
node_fund_custodian['label_custodian'] = 'FUND_CUSTODIAN'
node_fund_custodian.head()

,name,label_custodian
0,工商银行,FUND_CUSTODIAN
3,建设银行,FUND_CUSTODIAN
4,浙商银行,FUND_CUSTODIAN
5,农业银行,FUND_CUSTODIAN
6,招商银行,FUND_CUSTODIAN


#### 2.1.1.4 合并上市公司、基金管理公司以及基金托管公司的标签

In [42]:
node_companies = pd.merge(df_company_lsit_all,df_fund_company,on=['fullname','name'],how='outer').drop_duplicates()
node_companies = pd.merge(node_companies,node_fund_custodian,on=['name'],how='outer').drop_duplicates()
node_companies[node_companies.fullname.isnull()].name

19286    浦发银行/上海浦东发展银行
19287             广发银行
19288         中国邮政储蓄银行
19289             恒丰银行
Name: name, dtype: object

**<font color = red>问题十：我们可以发现有 4 家公司全称并没有拼上，分别是浦发银行/上海浦东发展银行，广发银行，中国邮政储蓄银行，恒丰银行。请你自行查询，将四个公司的全名补上，并将代码展示在下面：</font>**

In [43]:
node_companies.loc[node_companies.name == '浦发银行/上海浦东发展银行','fullname'] = '上海浦东发展银行股份有限公司'
node_companies.loc[node_companies.name == '广发银行','fullname'] = '广发银行股份有限公司'
node_companies.loc[node_companies.name == '中国邮政储蓄银行','fullname'] = '中国邮政储蓄银行股份有限公司'
node_companies.loc[node_companies.name == '恒丰银行','fullname'] = '恒丰银行股份有限公司'

In [44]:
node_companies[node_companies.fullname.isnull()].name

Series([], Name: name, dtype: object)

#### 2.1.1.5  处理标签

In [45]:
# 处理缺失值
node_companies['label_listed_company'] = node_companies['label_listed_company'].fillna('')
node_companies['label_manager'] = node_companies['label_manager'].fillna('')
node_companies['label_custodian'] = node_companies['label_custodian'].fillna('')

**<font color = red>问题十一：针对上面三种公司，我们在node_companies数据框中给出了不同的列分别作为它们的标签，但是这还不是理想的公司标签，我们应该把它汇总起来，现在请你为node_companies数据框再创建一列：node_companies['label']，将三种标签汇集到这一行来，并补充完整下面的代码块：</font>**

In [46]:
# 定义汇总标签函数
def join_label(ls):
    return 'COMPANY;'+ ';'.join([c for c in ls  if c !=''])

In [47]:
# 汇总标签
node_companies['label'] = node_companies.apply(lambda x: join_label([x['label_listed_company'],x['label_manager'],x['label_custodian']]),axis=1)

#### 2.1.1.6 抽取公司实体的数据，并创建头文件

In [48]:
# 基于company_lsit_all表，定义用于抽取公司实体所对应的相关信息的字典
all_dicts['node_companies'] = {  'fullname':'company:ID(company)', 
                            'name':'short_name', # 股票名称
                            'symbol':'share_code', # 股票全称
                            'market':'market', # 市场类型 （主板/中小板/创业板/科创板）
                            'exchange':'exchange', # 交易所代码
                            'list_status':'list_status', # 上市状态： L上市 D退市 P暂停上市
                            'list_date':'list_date:float', # 上市日期
                            'delist_date':'delist_date:float', # 退市日期
                            'setup_date':'setup_date',   # 成立日期
                            'label':':LABEL'
                           }

In [49]:
# 定义函数，用来基于上述字典，抽取公司实体所对应的相关信息，并创建头文件，该函数将重复用于后续的实体和关系
def generate_input_files(data,dicts,types,output_path='../data/output/'):
    cols = dicts.keys()
    temp = data[list(cols)]
    temp.drop_duplicates(cols,inplace=True)

    print('save header {}..')
    columns = ','.join(dicts.values())
    with open(output_path+'{}-header.csv'.format(types),'w') as f:
        f.write(columns)
    
    print('save_data ...\n',output_path+'{}.csv'.format(types))
    temp.to_csv(output_path+'{}.csv'.format(types),index=False,header=False)
    print(temp.shape)
    return temp

In [50]:
types = 'node_companies'
temp = generate_input_files(node_companies,dicts=all_dicts[types] ,types = types,output_path = output_path)

save header {}..
save_data ...
 ./data/output/node_companies.csv
(19290, 10)


### 2.1.2 行业
#### 2.1.2.1 为行业建立实体标签
**<font color = red>问题十二：请你观察下面代码块的输出，创建我们所需要的df_industry数据框。在这里提示你，这个数据框没有重复的行，并且一共81行。此外你还要再创建一列实体标签。请你补充完整下面的代码块：</font>**

In [51]:
df_industry = pd.DataFrame({'industry':[c for c in df_company_lsit_all.industry.unique() if not pd.isnull(c)]})
df_industry['label']='INDUSTRY'
df_industry.head()

,industry,label
0,货币金融服务,INDUSTRY
1,房地产业,INDUSTRY
2,软件和信息技术服务业,INDUSTRY
3,生态保护和环境治理业,INDUSTRY
4,铁路、船舶、航空航天和其它运输设备制造业,INDUSTRY


#### 2.1.2.2 抽取行业实体的数据，并创建头文件

In [52]:
# 定义用于抽取行业实体所对应的相关信息的字典
all_dicts['node_industries'] = {'industry':'industry:ID(industry)',
                                'label':':LABEL'
                               }

In [53]:
types = 'node_industries'
temp = generate_input_files(df_industry,dicts=all_dicts[types],types = types,output_path = output_path)

save header {}..
save_data ...
 ./data/output/node_industries.csv
(81, 2)


### 2.1.3 省份

In [54]:
df_company_detail['symbol'] = df_company_detail['ts_code'].apply(lambda x:x.split('.')[0])

#### 2.1.3.1 为省份建立实体标签

In [55]:
node_province1 = pd.DataFrame({'province':[c for c in df_company_detail.province.unique() if not pd.isnull(c)]})
node_province2 = pd.DataFrame({'province':[c for c in df_fund_company.province.unique() if not pd.isnull(c)]})
node_province = pd.concat([node_province1,node_province2]).drop_duplicates()
node_province['label']='PROVINCE'

#### 2.1.3.2 抽取行业实体的数据，并创建头文件

In [56]:
# 定义用于抽取省份实体所对应的相关信息的字典
all_dicts['node_province'] = {'province':'province:ID(province)',
                                'label':':LABEL'
                               }

In [57]:
types = 'node_province'
temp = generate_input_files(node_province,dicts=all_dicts[types],types = types, output_path = output_path)

save header {}..
save_data ...
 ./data/output/node_province.csv
(32, 2)


### 2.1.4 城市

#### 2.1.4.1 为城市建立实体标签
**<font color = red>问题十三：请你观察下面代码块的输出，创建我们所需要的node_city数据框，建议你从df_company_detail和df_fund_company两个数据库中获取城市的相关信息。请你并补充下面的代码块：</font>**

In [127]:
node_city1 = pd.DataFrame({'city':[c for c in df_company_detail.city.unique() if not pd.isnull(c)]})
node_city2 = pd.DataFrame({'city':[c for c in df_fund_company.city.unique() if not pd.isnull(c)]})
node_city = pd.concat([node_city1,node_city2]).drop_duplicates()

node_city['label']='CITY'
node_city

,city,label
0,徐州市,CITY
1,苏州市,CITY
2,潮州市,CITY
3,北京市,CITY
4,温州市,CITY
...,...,...
260,黑河市,CITY
262,蛟河市,CITY
263,兴宁市,CITY
267,巴彦淖尔市,CITY


#### 2.1.3.2 抽取城市实体的数据，并创建头文件

In [59]:
# 定义用于抽取城市实体所对应的相关信息的字典
all_dicts['node_city'] = {'city':'city:ID(city)',  
                          'label':':LABEL'
                          }

In [60]:
types = 'node_city'
temp = generate_input_files(node_city,dicts=all_dicts[types],types = types,output_path = output_path)

save header {}..
save_data ...
 ./data/output/node_city.csv
(342, 2)


### 2.1.5 人（上市公司董事高管）
对于公司高管，因为缺少人的唯一身份id，为了避免同名的影响，采用姓名+性别+生日进行hash生成唯一标识。

In [61]:
import hashlib

In [62]:
def get_md5(string):
    """Get md5 according to the string
    """
    byte_string = string.encode("utf-8")
    md5 = hashlib.md5()
    md5.update(byte_string)
    result = md5.hexdigest()
    return result

In [63]:
df_stk_managers['birthday'] = df_stk_managers['birthday'].apply(lambda x: -1 if pd.isnull(x) else int(x))
df_stk_managers['gender'] = df_stk_managers['gender'].apply(lambda x: -1 if x not in ['F','M'] else x)
df_stk_managers['hash_cust'] = df_stk_managers.apply(lambda x:get_md5('{}-{}-{}'.format(x['name'],x['gender'],x['birthday'])),axis=1 )

#### 2.1.5.1 为人建立实体标签

In [64]:
types = 'node_managers'

df_stk_managers['label'] = 'MANAGER'

#### 2.1.5.2 抽取人实体的数据，并创建头文件

In [65]:
# 定义用于抽取 人 实体所对应的相关信息的字典
all_dicts['node_managers'] = {'hash_cust':'manager_id:ID(manager_id)', 
                              'name':'name', 
                              'gender':'gender',
                              'edu':'edu',
                              'national':'national',
                              'birthday':'birthday:long',
                              'begin_date':'begin_date',
                              'end_date':'end_date',
                              'label':':LABEL'
                          }

In [66]:
node_managers = df_stk_managers[list(all_dicts[types].keys())]
node_managers = node_managers.sort_values(['name','gender','edu','national']).drop_duplicates('hash_cust',keep='first')

temp = generate_input_files(node_managers,dicts=all_dicts[types],types = types, output_path = output_path)

save header {}..
save_data ...
 ./data/output/node_managers.csv
(163570, 9)


### 2.1.5 基金

#### 2.1.5.1 数据预处理
* 对每个基金进行市场划分

In [67]:
df_fund_basic['fund'] = df_fund_basic['ts_code'].apply(lambda x: x.split('.')[0])
df_fund_basic['market'] = df_fund_basic['ts_code'].apply(lambda x: x.split('.')[1])

* 处理时间,我们将时间属性放于节点上

In [68]:
df_fund_basic['found_date'] = df_fund_basic['found_date'].apply(lambda x: -1 if pd.isnull(x) else int(x) )
df_fund_basic['due_date'] = df_fund_basic['due_date'].apply(lambda x: -1 if pd.isnull(x) else int(x) )
df_fund_basic['list_date'] = df_fund_basic['list_date'].apply(lambda x: -1 if pd.isnull(x) else int(x) )
df_fund_basic['issue_date'] = df_fund_basic['issue_date'].apply(lambda x: -1 if pd.isnull(x) else int(x) )
df_fund_basic['delist_date'] = df_fund_basic['delist_date'].apply(lambda x: -1 if pd.isnull(x) else int(x) )

* 转化数据类型

In [69]:
df_fund_basic['issue_amount'] = df_fund_basic['issue_amount'].apply(lambda x:float(x))
df_fund_basic['m_fee'] = df_fund_basic['m_fee'].apply(lambda x:float(x))
df_fund_basic['c_fee'] = df_fund_basic['c_fee'].apply(lambda x:float(x))
df_fund_basic['min_amount'] = df_fund_basic['min_amount'].apply(lambda x:float(x))
df_fund_basic['duration_year'] = df_fund_basic['duration_year'].apply(lambda x:float(x))

#### 2.1.5.2 为基金建立实体标签

In [70]:
types = 'node_funds'

df_fund_basic['label'] = 'FUND'

#### 2.1.5.3 抽取基金实体的数据，并创建头文件

In [71]:
# 定义用于抽取 基金 实体所对应的相关信息的字典
all_dicts['node_funds'] =  {'fund':'fund_code:ID(fund_code)', 
                            'name':'name', 
                            'fund_type':'fund_type', # 投资类型
                            'invest_type':'invest_type', # 投资风格
                            'type':'type',  # 基金类型 
                            'benchmark':'benchmark', # 业绩比较基准
                            'market':'market', #  场外'OF', 场内 'SH', 'SZ'
                            'found_date':'found_date:long',  # 成立日期
                            'delist_date':'delist_date:long',  # 退市日期
                            'status':'status',  #  存续状态: D摘牌 I发行 L已上市
                            'label':':LABEL' }

In [72]:
node_funds = df_fund_basic[list(all_dicts[types].keys())]

node_funds = node_funds.sort_values(['fund','found_date','delist_date'],ascending=False).drop_duplicates('fund',keep='first')

In [73]:
temp = generate_input_files(node_funds,dicts=all_dicts[types],types = types,output_path = output_path)

save header {}..
save_data ...
 ./data/output/node_funds.csv
(11187, 11)


## 2.2 关系

### 2.2.1 上市公司→行业

#### 2.2.1.1 为关系建立关系标签

In [74]:
rel_share_in_industry = df_company_lsit_all[['industry','fullname']]

rel_share_in_industry = rel_share_in_industry[rel_share_in_industry['fullname'].notnull()&rel_share_in_industry['industry'].notnull()]
rel_share_in_industry['type']='IN_INDUSTRY'

#### 2.2.1.2 抽取相关数据，并创建头文件

In [75]:
# 定义用于抽取上市公司→行业关系所对应的相关信息的字典
all_dicts['rel_share_in_industry'] = {'fullname':':START_ID(company)', # 股票代码
                                      'industry':':END_ID(industry)', # 所属行业
                                      'type':':TYPE'
                                      }

In [76]:
types = 'rel_share_in_industry'
temp = generate_input_files(rel_share_in_industry,dicts=all_dicts[types],types = types, output_path = output_path)

save header {}..
save_data ...
 ./data/output/rel_share_in_industry.csv
(4062, 3)


### 2.2.2 上市公司→城市
#### 2.2.2.1 为关系建立关系标签

In [77]:
rel_company_in_city1 = pd.merge(df_company_detail,df_company_lsit_all[['symbol','fullname']],on='symbol',how='left')[['fullname','city']]
rel_company_in_city2 = df_fund_company[['fullname','city']]
# 合并数据
rel_company_in_city = pd.concat([rel_company_in_city1,rel_company_in_city2]).drop_duplicates()

rel_company_in_city['type']='IN_CITY'

#### 2.2.2.2 抽取相关数据，并创建头文件

In [78]:
# 定义用于抽取上市公司→城市关系所对应的相关信息的字典
all_dicts['rel_company_in_city'] = {'fullname':':START_ID(company)', # 股票代码
                                  'city':':END_ID(city)', # 所属城市
                                  'type':':TYPE'
                                  }

In [79]:
rel_company_in_city = rel_company_in_city[rel_company_in_city['city'].notnull()&rel_company_in_city['fullname'].notnull()]

In [80]:
types = 'rel_company_in_city'
temp = generate_input_files(rel_company_in_city,dicts=all_dicts[types],types = types,output_path = output_path)

save header {}..
save_data ...
 ./data/output/rel_company_in_city.csv
(19251, 3)


### 2.2.3 城市 → 省份
#### 2.2.3.1 为关系建立关系标签
**<font color = red>问题十四：相信看了前面两个关系的构建，你已经比较了解如何创建关系文件。在这里请你观察下面代码块的输出结果，并且基于df_company_detail和df_fund_company里面的城市、省份信息，构建一个合适的rel_city_in_province。请你并补充下面的代码块：</font>**

In [128]:
rel_city_in_province1= df_company_detail[['province','city']]
rel_city_in_province2= df_fund_company[['province','city']]
rel_city_in_province = pd.concat([rel_city_in_province1,rel_city_in_province2]).drop_duplicates()

rel_city_in_province = rel_city_in_province[rel_city_in_province['province'].notnull()&rel_city_in_province['city'].notnull()]
rel_city_in_province['type']='IN_PROVINCE'
rel_city_in_province

,province,city,type
0,江苏,徐州市,IN_PROVINCE
1,江苏,苏州市,IN_PROVINCE
2,广东,潮州市,IN_PROVINCE
3,北京,北京市,IN_PROVINCE
4,浙江,温州市,IN_PROVINCE
...,...,...,...
24325,黑龙江,黑河市,IN_PROVINCE
24448,吉林,蛟河市,IN_PROVINCE
24477,广东,兴宁市,IN_PROVINCE
25023,内蒙古,巴彦淖尔市,IN_PROVINCE


#### 2.2.3.2 抽取相关数据，并创建头文件

In [82]:
# 定义用于抽取城市→省份关系所对应的相关信息的字典
all_dicts['rel_city_in_province'] = {'city':':START_ID(city)', 
                                  'province':':END_ID(province)', 
                                  'type':':TYPE'
                                  }
types = 'rel_city_in_province'

temp = generate_input_files(rel_city_in_province,dicts=all_dicts[types],types = types,output_path = output_path)

save header {}..
save_data ...
 ./data/output/rel_city_in_province.csv
(342, 3)


### 2.2.4 上市公司 → 人(董事高管)

#### 2.2.4.1 定义关系标签，处理数据
基于数据，我们可以将高管关系分为以下六类，直接使用type作为标签：
- 董事会成员
- 监事
- 委员会成员
- 高管
- 核心技术人员
- 其他

In [83]:
# 处理证券代码
df_stk_managers['symbol'] = df_stk_managers['ts_code'].apply(lambda x:x.split('.')[0])
# 处理时间数据
df_stk_managers['begin_date'] = df_stk_managers['begin_date'].apply(lambda x: -1 if pd.isnull(x) else int(x) )
df_stk_managers['end_date'] = df_stk_managers['end_date'].apply(lambda x: -1 if pd.isnull(x) else int(x))
df_stk_managers['ann_date'] = df_stk_managers['ann_date'].apply(lambda x: -1 if pd.isnull(x) else int(x))

In [84]:
rel_listed_company_has_manager = pd.merge(df_stk_managers,df_company_lsit_all[['symbol','fullname']],on='symbol',how='left')

In [85]:
types = 'rel_listed_company_has_manager'

rel_listed_company_has_manager = rel_listed_company_has_manager[rel_listed_company_has_manager['fullname'].notnull()&rel_listed_company_has_manager['hash_cust'].notnull()]

In [86]:
rel_listed_company_has_manager = rel_listed_company_has_manager.sort_values(['fullname','hash_cust','title','begin_date','end_date'],ascending=False).drop_duplicates(['fullname','hash_cust','title','begin_date'],keep='first')

#### 2.2.4.2 抽取相关数据，并创建头文件
**<font color = red>问题十五：请你观察比较前面关系构建中的字典，思考在下面代码块应该补充什么。如果你没有思路的话，可以看2.2.4.1节下面的那句话，可能会有所启发。最后，请你并补充下面的代码块：</font>**

In [87]:
# 定义用于抽取上市公司→人关系所对应的相关信息的字典
all_dicts['rel_listed_company_has_manager'] = {'fullname':':START_ID(company)', 
                                              'hash_cust':':END_ID(manager_id)', 
                                              'begin_date':'begin_date:long',
                                              'end_date':'end_date:long',
                                              'title':'title',
                                              'lev':':TYPE'#补充该部分：使用type做标签
                                              }

In [88]:
temp0 = generate_input_files(rel_listed_company_has_manager,dicts=all_dicts[types],types = types,output_path=output_path)

save header {}..
save_data ...
 ./data/output/rel_listed_company_has_manager.csv
(428985, 6)


### 2.2.5 基金 → 托管人
#### 2.2.5.1 处理数据，为关系建立关系标签

In [89]:
# 合并相关数据
temp = node_companies[['name','fullname']].rename(columns = {'name':'custodian'})
rel_fund_has_custodian = pd.merge(df_fund_basic,temp, on='custodian',how='left')[['fund','fullname']]

In [90]:
rel_fund_has_custodian['type']='HAS_CUSTODIAN'

#### 2.2.5.2 抽取相关数据，并创建头文件

In [91]:
# 定义用于抽取基金→托管人关系所对应的相关信息的字典
all_dicts['rel_fund_has_custodian'] = {'fund':':START_ID(fund_code)',   # 基金代码
                                      'fullname':':END_ID(company)',   # 托管人
                                      'type':':TYPE'
                                     }

In [92]:
types = 'rel_fund_has_custodian'

temp = generate_input_files(rel_fund_has_custodian,dicts=all_dicts[types],types = types,output_path = output_path)

save header {}..
save_data ...
 ./data/output/rel_fund_has_custodian.csv
(11253, 3)


In [93]:
rel_fund_has_custodian.head().T

,0,1,2,3,4
fund,159821,159822,588080,588090,511000
fullname,中国工商银行股份有限公司,中国工商银行股份有限公司,中国工商银行股份有限公司,中国建设银行股份有限公司,浙商银行股份有限公司
type,HAS_CUSTODIAN,HAS_CUSTODIAN,HAS_CUSTODIAN,HAS_CUSTODIAN,HAS_CUSTODIAN


### 2.2.6 基金 → 管理人
#### 2.2.6.1 处理数据，为关系建立关系标签

In [94]:
# 合并相关数据
temp = node_companies[['name','fullname']].rename(columns = {'name':'management'})
rel_fund_has_management = pd.merge(df_fund_basic,temp, on='management',how='left')[['fund','fullname']]

In [95]:
rel_fund_has_management['type']='HAS_MANAGEMENT'

#### 2.2.6.2 抽取相关数据，并创建头文件

In [96]:
# 定义用于抽取基金→管理人关系所对应的相关信息的字典
all_dicts['rel_fund_has_management'] = {'fund':':START_ID(fund_code)',   # TS基金代码
                                      'fullname':':END_ID(company)',   # 股票代码
                                      'type':':TYPE'
                                     }

In [97]:
types = 'rel_fund_has_management'

temp = generate_input_files(rel_fund_has_management,dicts=all_dicts[types],types = types,output_path = output_path)

save header {}..
save_data ...
 ./data/output/rel_fund_has_management.csv
(11270, 3)


In [98]:
rel_fund_has_management.head().T

,0,1,2,3,4
fund,159821,159822,588080,588090,511000
fullname,中银国际证券股份有限公司,银华基金管理股份有限公司,易方达基金管理有限公司,华泰柏瑞基金管理有限公司,招商基金管理有限公司
type,HAS_MANAGEMENT,HAS_MANAGEMENT,HAS_MANAGEMENT,HAS_MANAGEMENT,HAS_MANAGEMENT


### 2.2.7 基金 → 上市公司（基金持仓）
#### 2.2.7.1 处理数据，为关系建立关系标签

In [99]:
df_fund_portfolio['fund'] = df_fund_portfolio['ts_code'].apply(lambda x: x.split('.')[0])

In [100]:
df_fund_portfolio['symbol'] = df_fund_portfolio['symbol'].apply(lambda x: x.split('.')[0])

In [101]:
df_fund_portfolio['ann_date'] = df_fund_portfolio['ann_date'].apply(lambda x: -1 if pd.isnull(x) else int(x) )
df_fund_portfolio['end_date'] = df_fund_portfolio['end_date'].apply(lambda x: -1 if pd.isnull(x) else int(x) )

In [102]:
# 合并相关数据
rel_fund_listed_company_portfolio = pd.merge(df_fund_portfolio,df_company_lsit_all[['symbol','fullname']],on='symbol',how='left')

In [103]:
rel_fund_listed_company_portfolio['type']='IN_PORTFOLIO'

#### 2.2.7.2 抽取相关数据，并创建头文件

In [104]:
# 定义用于抽取基金→上市公司关系所对应的相关信息的字典
all_dicts['rel_fund_listed_company_portfolio'] = {'fund':':START_ID(fund_code)',   # TS基金代码
                                                  'fullname':':END_ID(company)',   # 股票代码
                                                  'ann_date':'ann_date:float',   # 公告日期
                                                  'end_date':'end_date:float' ,  # 截止日期
                                                  'mkv':'mkv:float' ,  # 持有股票市值(元)
                                                  'amount':'amount:float' ,  # 持有股票数量（股）
                                                  'stk_mkv_ratio':'stk_mkv_ratio:float',   # 占股票市值比
                                                  'stk_float_ratio':'stk_float_ratio:float',   # 占流通股本比例
                                                  'type':':TYPE'
                                                 }

In [105]:
rel_fund_listed_company_portfolio = rel_fund_listed_company_portfolio.sort_values(['fund','fullname','ann_date','end_date'],ascending=False).drop_duplicates(['fund','fullname'],keep='first')

In [106]:
df_fund_portfolio.shape

(1235280, 9)

In [107]:
types = 'rel_fund_listed_company_portfolio'

temp = generate_input_files(rel_fund_listed_company_portfolio,dicts=all_dicts[types],types = types,output_path = output_path)

save header {}..
save_data ...
 ./data/output/rel_fund_listed_company_portfolio.csv
(526357, 9)


## 3 Neo4j图数据库安装和知识图谱构建

### 3.1 安装程序软件JAVA，Neo4j和插件apoc
注意选择合适的JAVA版本进行安装，注意Neo4j和插件apoc的版本需要匹配，以下搭配就是一个互相兼容的例子 (百度网盘链接：https://pan.baidu.com/s/1kjYuEnxtiTE22VLXF-RlYA 提取码：2021)，你也可以从以下三个网页中寻找并下载适合你的电脑的其他版本：   
* JAVA    
https://adoptopenjdk.net/releases.html?variant=openjdk8&jvmVariant=hotspot    
* Neo4j    
https://neo4j.com/download-center/#community   
* apoc插件    
https://github.com/neo4j-contrib/neo4j-apoc-procedures/releases/3.5.0.11

### 3.2 添加Java和Neo4j的环境变量
### 3.3 在cmd中运行Neo4j
> neo4j console

### 3.4 根据cmd页面提示，打开Neo4j的web界面
一般为：http://localhost:7474/   
在Neo4j的web界面进行操作可以得到更好的可视化效果
### 3.5 在python中实现Neo4j交互

In [117]:
import os
from py2neo import Graph
import warnings
warnings.filterwarnings('ignore')

In [118]:
graph = Graph(
    "bolt://localhost:7687",
    usernmae="neo4j",# 输入默认账号和密码，在python登入Neo4j
    password="neo4j"
)

### 3.6 导入相关数据构建知识图谱
在cmd命令行中输入以下generate_scripts函数的输出结果，注意此操作必须关闭Neo4j进行。

In [110]:
import os
file_ls = os.listdir('./data/output')
node_ls =  [file for file in file_ls if file.startswith('node')]
relationship_ls =  [file for file in file_ls if file.startswith('rel')]

In [111]:
def generate_scripts(path,database='graph.db'):
    file_ls = os.listdir(path)
    node_ls =  [file for file in file_ls if file.startswith('node')]
    relationship_ls =  [file for file in file_ls if file.startswith('rel')]

    print("neo4j-admin import --database={}  ".format(database))
    for i in range(int(len(node_ls)/2)):
        print("--nodes {},{}  ".format(node_ls[2*i],node_ls[2*i+1]))

    for i in range(int(len(relationship_ls)/2)):
        print("--relationships {},{}  ".format(relationship_ls[2*i],relationship_ls[2*i+1]))
    
    print("--ignore-duplicate-nodes=true  ")
    print("--ignore-missing-nodes=true")    

In [112]:
generate_scripts(path='./data/output',database='finance_graph.db')

neo4j-admin import --database=finance_graph.db  
--nodes node_city-header.csv,node_city.csv  
--nodes node_companies-header.csv,node_companies.csv  
--nodes node_funds-header.csv,node_funds.csv  
--nodes node_industries-header.csv,node_industries.csv  
--nodes node_managers-header.csv,node_managers.csv  
--nodes node_province-header.csv,node_province.csv  
--relationships rel_city_in_province-header.csv,rel_city_in_province.csv  
--relationships rel_company_in_city-header.csv,rel_company_in_city.csv  
--relationships rel_fund_has_custodian-header.csv,rel_fund_has_custodian.csv  
--relationships rel_fund_has_management-header.csv,rel_fund_has_management.csv  
--relationships rel_fund_listed_company_portfolio-header.csv,rel_fund_listed_company_portfolio.csv  
--relationships rel_listed_company_has_manager-header.csv,rel_listed_company_has_manager.csv  
--relationships rel_share_in_industry-header.csv,rel_share_in_industry.csv  
--ignore-duplicate-nodes=true  
--ignore-missing-nodes=true


### 3.7 创建知识图谱数据库的软链接
由于Neo4j默认显示graph.db数据库，我们需要将finance_graph.db链接到graph.db上来，此时需要用到应用程序junction.exe，来源于网页https://docs.microsoft.com/zh-cn/sysinternals/downloads/junction
注意，此时需要删除databases文件夹中的graph.db文件，并在cmd命令行中输入：   
> cd C:\Program Files\neo4j-community-3.5.14-windows\neo4j-community-3.5.14\data\databases      
> junction -s  graph.db finance_graph.db     

最后，在cmd命令行中重新运行Neo4j软件，重新打开3.4节中的网址，如果看到的页面类似下面的图，那么我们的数据库就构建成功了。

![title](img/初始页面.jpg)

### 3.8 为知识图谱创建索引以提高查询效率

In [119]:
index_scripts = """CREATE INDEX ON :CITY(city);
                CREATE INDEX ON :FUND(fund_code);
                CREATE INDEX ON :COMPANY(company);
                CREATE INDEX ON :FUND_CUSTODIAN(company);
                CREATE INDEX ON :FUND_MANAGER(company);
                CREATE INDEX ON :LISTED_COMPANY(company);
                CREATE INDEX ON :INDUSTRY(industry);
                CREATE INDEX ON :MANAGER(manager_id);
                CREATE INDEX ON :PROVINCE(province);
                CREATE INDEX ON :LISTED_COMPANY(share_code);
                """

In [120]:
for scripts in index_scripts.split('\n'):
    scripts = scripts.strip()
    if len(scripts)>5 :
        print(scripts) # 该处输出也可以直接在Neo4j页面的命令行输入，可以实现同样的功能
        result = graph.run(scripts)

CREATE INDEX ON :CITY(city);
CREATE INDEX ON :FUND(fund_code);
CREATE INDEX ON :COMPANY(company);
CREATE INDEX ON :FUND_CUSTODIAN(company);
CREATE INDEX ON :FUND_MANAGER(company);
CREATE INDEX ON :LISTED_COMPANY(company);
CREATE INDEX ON :INDUSTRY(industry);
CREATE INDEX ON :MANAGER(manager_id);
CREATE INDEX ON :PROVINCE(province);
CREATE INDEX ON :LISTED_COMPANY(share_code);


## 4 知识图谱的应用

知识图谱应用部分主要从三个方面展开：
* 信息可视化展示
* 关联查询
* 投研

我们将利用 Neo4j 的查询语言 `Cypher` 进行相关查询。此部分推荐在Neo4j的web界面进行调试，可视化效果更佳。
### 4.1 湖北国资概念股的信息可视化查询

**<font color = red>问题十六：我们知道东湖高新、凯龙股份、襄阳轴承、ST双环、广济药业、楚天高速是6家湖北国资概念股，请你可视化展现这6家公司在省份、城市和行业上的分布情况，请你将查询语言补到下方代码块中，再插入图片展示你的可视化结果。</font>**

In [ ]:
MATCH p1 = (a:LISTED_COMPANY)-[:IN_INDUSTRY]-(b:INDUSTRY)
    WHERE a.share_code in ["000707","002783","000952","600133","000678","600035"]
MATCH p2 = (a)-[:IN_CITY]-(c:CITY)-[:IN_PROVINCE]-(d:PROVINCE)
RETURN p1,p2

![title](img/湖北国资.png)

### 4.2 江苏省最受基金欢迎的上市公司
**<font color = red>问题十七：请你观察下面代码块的输出，在query1语句中补充完成以下两个功能：1.创建列名city，company，num，2.将数据按照num降序排列，注意，请你删去下方代码块中的#号部分再填写所需内容（#号代表该地方需要填充），否则程序将会报错：</font>**

In [121]:
query1 = """
MATCH p = (a:FUND)-[:IN_PORTFOLIO]->(b:LISTED_COMPANY)-[:IN_CITY]-(c:CITY)-[:IN_PROVINCE]->(d:PROVINCE {province:"江苏"})  
RETURN c.city as city,b.share_code as code, b.company as company,count(DISTINCT a.fund_code) as num
ORDER BY num DESC
limit 10
"""

df = graph.run(query1).to_data_frame()
df.head(10)

,city,code,company,num
0,无锡市,300842,无锡帝科电子材料股份有限公司,1216
1,连云港市,600276,江苏恒瑞医药股份有限公司,1144
2,南京市,601688,华泰证券股份有限公司,963
3,无锡市,300782,江苏卓胜微电子股份有限公司,958
4,南京市,688377,南京迪威尔高端制造股份有限公司,841
5,无锡市,603259,无锡药明康德新药开发股份有限公司,772
6,宿迁市,002304,江苏洋河酒厂股份有限公司,702
7,南京市,601009,南京银行股份有限公司,674
8,常州市,603655,常州朗博密封科技股份有限公司,653
9,苏州市,603283,苏州赛腾精密电子股份有限公司,652


### 4.3 上市公司“独立性”审查
**<font color = red>问题十八：我们认为如果一个人同时出2家以上的上市公司中担任“董事会成员”、“监事”、“委员会成员”，“高管”，“核心技术人员”等职位，即是涉及“兼任”的情况。“兼任”使得两家以上的上市公司可能被同一个人控制，这可能使得“独立性”产生问题。现在请你调查“澳洋科技”这家上市公司董事会成员的情况，再看看他们在公司外的兼任情况，请你将查询语言补到代码块中，再插入图片展示你的可视化结果：</font>**

In [ ]:
MATCH (a:LISTED_COMPANY)-[r1:董事会成员]->(b:MANAGER)
    WHERE a.share_code in ['002172'] and r1.end_date = -1
RETURN a,b

![title](img/澳洋科技现任.png)

In [ ]:
MATCH (a:LISTED_COMPANY)-[r1:董事会成员]->(b:MANAGER)
    WHERE a.share_code in ['002172'] and r1.end_date = -1
MATCH  p1 = (c:COMPANY)-[r1:其他|监事|高管|委员会成员|董事会成员|核心技术人员]->(b)
MATCH  p2 = (e:COMPANY)-[r2:其他|监事|高管|委员会成员|董事会成员|核心技术人员]->(b)
    WHERE  e.company<>c.company
RETURN p1, p2

![title](img/独立性.png)

### 4.4 持仓相似基金推荐
**<font color = red>问题十九：我们想要根据基金景顺资源(162607.SZ)持仓在不同行业的分布情况，推荐与其持仓行业最相似的五个基金，以下是其查询语言，请你给出对主干语言作用的阐述。通过和前面问题的比对，相信你很容易发现图数据库的优势所在。</font>**

In [116]:
fund_code = "162607"

query3 = """
MATCH p = (a:FUND)-[:IN_PORTFOLIO]->(b:COMPANY)-[:IN_INDUSTRY]-(c:INDUSTRY)  
    WHERE a.fund_code="{}"   
WITH  COLLECT(DISTINCT c.industry) as industry_base  
MATCH  (d:FUND)-[:IN_PORTFOLIO]->(e:COMPANY)-[:IN_INDUSTRY]-(f:INDUSTRY) 
WITH d.fund_code as fund_code, industry_base,collect(DISTINCT f.industry) as industry_target 
RETURN fund_code,
       toFloat(length([ind in industry_target where ind in industry_base ]))/length(industry_target) as ration
ORDER BY ration DESC
LIMIT 5
""".format(fund_code)

df = graph.run(query3).to_data_frame()

df.head()

,fund_code,ration
0,007153,1.0
1,004597,1.0
2,004598,1.0
3,008298,1.0
4,007154,1.0


**A**：首先，根据要求写一个匹配语句，记录景顺资源基金持仓持仓情况，以及其对应的行业结构，并用industry_base变量记录下来。接着再写一个匹配语句，将具有所有基金的行业结构也表示出来，计算它们与景顺资源的行业结构的相似程度，即ration指标，输出和景顺资源行业结构最相似（ration降序排列）的前5个基金（LIMIT为5）的名称以及相似度。

### 4.5 股票推荐
**<font color = red>问题二十：你认为疫情之下，医药股大有可为，你想要在近期表现颇佳的000336.OF、001679.OF、002939.OF和001643.OF四只公募基金中找出它们配置的医药制造业股票进行研究。请你将查询语言补到代码块中，再插入图片展示你的可视化结果：</font>**

In [ ]:
MATCH p = (a:FUND)-[:IN_PORTFOLIO]->(b:COMPANY)-[:IN_INDUSTRY]->(c:INDUSTRY)
 WHERE a.fund_code in ["000336","001679","002939","001643"] and c.industry = "医疗保健"
RETURN p

![title](img/医疗保健.png)

# 5 知识图谱的扩展
**<font color = red>附加题：你能否扩展现有的资本市场知识图谱？如果可以，请找出对应的数据来源，并指出所蕴含的新的实体和关系是什么。 </font>** 